In [48]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
from module.apps import App
import ipywidgets as widgets
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
pluie_obs = pd.read_csv(
    "Selle_et_Morvillers/Pluv_Moy_9_Stat_Selle_1985_2003.prn",
    delim_whitespace=True,
    parse_dates=True,
    dayfirst=True,
    index_col=0
)

etp_obs = pd.read_csv(
    "Selle_et_Morvillers/Etp_Moy_9_Stat_Selle_1985_2003.prn",
    delim_whitespace=True,
    parse_dates=True,
    dayfirst=True,
    index_col=0
)

debit = pd.read_csv(
    "Selle_et_Morvillers/Debit_Selle_Plachy_1985_2003.prn",
    delim_whitespace=True,
    parse_dates=True,
    dayfirst=True,
    index_col=0,
)['Debit']
debit = debit.replace(-2, np.nan)

niveau = pd.read_csv(
    "Selle_et_Morvillers/Niv_Morvillers_00608X0028_1985_2003.prn",
    delim_whitespace=True,
    parse_dates=True,
    dayfirst=True,
    index_col=0,
)['Niveau']
niveau = niveau.replace(9999., np.nan)

In [8]:
app = App(debit, niveau)
app.set_meteo(pluie_obs, etp_obs)
column1 = widgets.VBox(
    [
        widgets.Label("Paramètres", style=dict(font_weight="bold", background='lightblue')),
        app.widgetA,
        app.widgetR,
        app.widgetTHG,
        app.widgetTG1,
        app.widgetbaseNiv,
        app.widgetcoeffEmmag,
        widgets.Label("Critères", style=dict(font_weight="bold", background='lightblue')),
        app.textNash,
        app.textKGE,
        app.textNashNiv
    ],
    layout=widgets.Layout(width='35%')
)

column2 = widgets.VBox(
    [
        app.btnOpti,
        app.fig.canvas,
    ]
)

widgets.HBox([column1, column2])

On fait tourner les modèles avec le jeu de paramètres issu du calage.

In [14]:
from module.utils import drias2020
def run_simulations(b):
    btn.disabled = True
    progress.value = 0
    for gcm, rcm, period in drias2020:
        pluie = 'sims_cc/{0}_{1}_{2}/Pluie_Selle_et_Morvillers_{0}_{1}_{2}'.format(
            gcm,
            rcm,
            period
        )
        pluie = pd.read_csv(
            pluie,
            delim_whitespace=True,
            parse_dates=True,
            dayfirst=True,
            index_col=0
        )
        etp = 'sims_cc/{0}_{1}_{2}/ETP_Selle_et_Morvillers_{0}_{1}_{2}'.format(
            gcm,
            rcm,
            period
        )
        etp = pd.read_csv(
            etp,
            delim_whitespace=True,
            parse_dates=True,
            dayfirst=True,
            index_col=0
        )
        app.set_meteo(pluie, etp)
        app.run(-9999)
        os.makedirs('results_cc', exist_ok=True)
        app.result.to_csv(
            'results_cc/{0}_{1}_{2}.csv'.format(
                gcm, rcm, period
            ),
        )
        progress.value += 1
    app.set_meteo(pluie_obs, etp_obs)
    btn.disabled = False
label = widgets.Label("Lancer les simulations :")
btn = widgets.Button(
    description='Lancer',
    disabled=False,
    icon='check'
)
btn.on_click(run_simulations)
progress = widgets.IntProgress(
    value=0,
    min=0,
    max=len(drias2020),
    #description='Chargement : ',
    bar_style='info', # 'success', 'info', 'warning', 'danger' or ''
    #style={'bar_color': 'blue'},
    orientation='horizontal'
)
widgets.VBox([label, widgets.HBox([btn, progress])])

In [85]:
def main_sim(b):
    select = check_sim()
    df = {}
    for sel, f in select.items():
        df[tuple(sel.split('_'))] = pd.read_csv(f, parse_dates=True, index_col=0)
    df = pd.concat(df, axis=1, names=['Modèle', 'RCP', 'Variable'])
    df = df.xs(key=selectVariable.value, level='Variable', axis=1)
    df = transform_indic(df)
    plot_sim(df)

def transform_indic(df):
    indic = selectIndic.value
    if indic == 'Moyenne mensuelle':
        df = df.resample('M').mean()
    elif indic == 'Minimum mensuel':
        df = df.resample('M').min()
    elif indic == 'Maximum mensuel':
        df = df.resample('M').max()
    elif indic == 'Moyenne annuelle':
        df = df.resample('M').mean()
    elif indic == 'Minimum annuel':
        df = df.resample('Y').min()
    elif indic == 'Maximum annuel':
        df = df.resample('Y').max()
    return df

def plot_sim(df):
    ax.clear()
    df.plot(ax=ax, legend=False, grid=True)
    legend = ax.legend(loc="upper left", bbox_to_anchor=(1.02, 0, 0.07, 1), prop={'size':6})
    fig.tight_layout()

def check_sim():
    models = selectModels.value
    rcps = selectRcps.value
    select = {}
    for model in models:
        gcm, rcm = model.split(' / ')
        for rcp in rcps:
            f = 'results_cc/{0}_{1}_{2}.csv'.format(gcm, rcm, rcp)
            if os.path.exists(f):
                select['{0} / {1}_{2}'.format(gcm, rcm, rcp)] = f
    return select



models = ['{0} / {1}'.format(model[0], model[1]) for model in drias2020]
models = list(set(models))
models.sort()
selectModels = widgets.SelectMultiple(
    options=models,
    description='Modèles :',
    disabled=False
)
selectRcps = widgets.SelectMultiple(
    options=['rcp2.6', 'rcp4.5', 'rcp8.5'],

    description='RCPs :',
    disabled=False
)
selectVariable = widgets.Dropdown(
    options=['Débit', 'Niveau'],
    description='Variable :',
    disabled=False
)
selectIndic = widgets.Dropdown(
    options=[
        'Journalier',
        'Moyenne mensuelle',
        'Minimum mensuel',
        'Maximum mensuel',
        'Moyenne annuelle',
        'Minimum annuel',
        'Maximum annuel'
    ],
    description='Indicateur :',
    disabled=False
)
selectPlot = widgets.Dropdown(
    options=['Spaghetti', 'Q5/Mediane/Q95', 'Min/Moy/Max'],
    description='Tracé :',
    disabled=False
)

checkAnom = widgets.Dropdown(
    options = ['Non', '2021-2050', '2041-2070', '2071-2100'],
    description = 'Anomalie',
)
btn = widgets.Button(
    description='Tracer',
    disabled=False,
    icon='check'
)
btn.on_click(main_sim)
col_para = widgets.VBox([selectModels, selectRcps, selectVariable, selectIndic, selectPlot, checkAnom, btn])
plt.ioff()
fig, ax = plt.subplots(1)
fig.canvas.header_visible = False
plt.ion()
widgets.HBox([col_para, fig.canvas])
